In [ ]:
# set up our local code path
import sys
sys.path.append('/home/serenity/dev/shadows/serenity/serenity-mdrecorder/src')

# standard imports
import pandas as pd
import plotly.graph_objects as go

from cloudwall.serenity.mdrecorder.tickstore import LocalTickstore
from datetime import datetime as dt
from pathlib import Path
from plotly.subplots import make_subplots

# set up Cufflinks
import cufflinks as cf
cf.set_config_file(theme='ggplot',sharing='public',offline=True)

In [ ]:
# Connect to Behemoth
phemex_tickstore = LocalTickstore(Path('/behemoth/db/PHEMEX_TRADES'), 'time')


In [ ]:
phemex_trades = tickstore.select('BTCUSD', start=dt(2020, 4, 1), end=dt(2020, 12, 31))

In [ ]:
del phemex_trades['product_id']
del phemex_trades['sequence']
del phemex_trades['side']
phemex_trades['price'] = phemex_trades['price'] / 10000
phemex_trades.head()

In [ ]:
cbp_tickstore = LocalTickstore(Path('/behemoth/db/COINBASE_PRO_TRADES'), 'time')
cbp_trades = cbp_tickstore.select('BTC-USD', start=dt(2020, 4, 1), end=dt(2020, 12, 31))
del cbp_trades['product_id']
del cbp_trades['sequence']
del cbp_trades['trade_id']
del cbp_trades['side']

In [ ]:
cbp_5m = cbp_trades['price'].resample('5Min').ohlc()

In [ ]:
phemex_5m = phemex_trades['price'].resample('5Min').ohlc()

In [ ]:
fig = go.Figure()

# Add traces
fig.add_trace(
    go.Scatter(x=phemex_5m.index, y=phemex_5m['close'], name="Phemex")
)


fig.add_trace(
    go.Scatter(x=cbp_5m.index, y=cbp_5m['close'], name="Coinbase Pro")
)

# Add figure title
fig.update_layout(
    title_text="Phemex futures vs. Coinbase spot (5m)"
)

fig.show()